# Why use classes?

Classes facilitate the two core tenets of code design: **code re-use** and
**encapsulation**:
* **Code re-use** is facilitated by:
    - Multiple instantiation. From the same class you can create multiple
      instances that all re-use the class methods.
    - Inheritance. One class can re-use methods from a parent class.
* **Encapsulation**.
    - Class methods and variables are encapsulated in the class namespace and
      are not accessible in the global namespace.
    - Instance methods and variables are encapsulated in an instance-specific
      namespace and are not accessible in the class or the global namespace.
    
By the way, even if you don't write object-oriented code, you still use classes
and instances all the time because under the hood, everything in python is an
instance of the built-in class 'object'. You can check this by running
`isinstance(x, object)` --- for any `x`, such as an integer, function, list,
etc. that will return True.

# Namespaces

A namespace is a mapping from variable names to objects. You can think of it as
a dictionary. An object can be looked up from a namespace via dot-access with
the namespace name --- for example, `my_namespace.my_variable` refers to the
object associated with the key `'my_variable'` in the namespace `my_namespace`.

There are usually multiple namespaces when you run something. Here are some
examples:
* Python has a built-in namespace (this one is actually special in that it
  doesn't need to be explicitly accessed) containing names like `return`, `for`,
  `def`, etc.
* Importing modules creates namespaces. For example, `import numpy as np`
  creates a `numpy` namespace and aliases it as `np` in the current namespace in
  which the import statement is. The `numpy` namespace contains all variables
  defined in the `numpy` module (specifically, the variables defined in
  `numpy`'s `__init__.py` file).
* When a function is entered, it creates a temporary namespace that is removed
  when the function is exited. More on this later.
* Classes are namespaces. More on this later.

Namespaces can be nested. For example, this happens for nested modules (e.g.
`np.linalg.norm` is accessing a variable in a nested namespace) and for nested
functions.

You can think of python execution as being in a namespace at every point in
time. This may be the namespace of the current module being run, a temporary
function namespace (if the interpreter happens to be inside a function at the
moment), or the namespace of some other imported module (if the interpreter is
inside that module at the moment).

In general, it is good to keep namespaces organized. That is why wildcard
imports such as `from numpy import *` are bad practice --- they clutter the
current module's namespace, which can make it hard to tell where variables are
coming from.

In [17]:
# When the interpreter is in a nested namespace, variables in the outer
# namespace are accesible. Here we see that the global namespace is accessible
# when the interpreter is inside a temporary function namespace:

%reset -f

global_string = 'global_string'

def f():
    print('f() can access ' + global_string)
    
f()
print('Parent scope can still access ' + global_string)

# Consequently, nested functions create nested temporary namespaces.

f() can access global_string
Parent scope can still access global_string


In [21]:
# Danger: Since the outer namespace is accessible, mutable objects in the outer
# namespace can be modified when the interpreter is in the inner namespace.

%reset -f

global_list = ['global_string']

def f():
    global_list.pop()
    global_list.append('corruption')
    
print('global_list in outer scope is: {}'.format(global_list))
f()
print('Now global_list in outer scope is: {}'.format(global_list))

# For this reason, try to make sure variables in outer namespaces (e.g. a
# module namespace) have an immutable type when possible.

global_list in outer scope is: ['global_string']
Now global_list in outer scope is: ['corruption']


In [22]:
# Variables in temporary function namespace are not accesible after the funtion
# exits because the temporary function namespace is removed then.

%reset -f

def f():
    local_string = 'local_string'
    print('f() can access ' + local_string)
    
f()

try:
    # This raises error because outer scope cannot access local_string
    print(local_string)
except Exception as e:
    print(e)

f() can access local_string
name 'local_string' is not defined


In [26]:
# Overriding an outer namespace's variable in an inner namespace will create a
# new variable in the inner namespace without affecting the variable in the
# outer namespace.

%reset -f

global_string = 'global_string'

def f():
    global_string = 'locally_overriden_global_string'
    print('f() sees: ' + global_string)
    
f()
print('Parent scope sees: ' + global_string)

# In general, when looking up variables, python will look in the current
# namespace (the one that the exceution is in at the moment) first. Only if the
# variable name is not found in the current namespace will python check the
# containing namespace and go up from there.

f() sees: locally_overriden_global_string
Parent scope sees: global_string


In [35]:
# The `global` keyword can be used to prevent local overriding

%reset -f

global_string = 'global_string'

def f():
    global global_string
    global_string = 'locally_overriden_global_string'
    print('f() sees: ' + global_string)
    
f()
print('Parent scope sees: ' + global_string)

f() sees: locally_overriden_global_string
Parent scope sees: global_string
Parent scope sees: locally_overriden_global_string


In [39]:
# The `global` keyword can also be used to define a variable in the global scope

%reset -f

def f():
    global local_string
    local_string = 'local_string'
    
f()
print('Parent scope sees: ' + local_string)

Parent scope sees: local_string


# Classes

In [29]:
# Like functions, classes create namespaces contained in the module namespace of
# the file that the class is in. However, unlike temporary function namespaces,
# class namespaces are persistent. Hence, like modules, variables in these
# namespaces can be dot-accessed by the class name. Also, unlike a function a
# class is not executed itself, so the execution never enters the class
# namespace like it does a function's temporary namespace.

%reset -f

class MyClass():
    class_string = 'class_string'
    
    def my_method():
        print('MyClass can access ' + MyClass.class_string +
              ' as an attribute of MyClass')
        
    # Note: Methods in MyClass cannot access class_string directly because
    # execution never enters a class's namespace. When MyClass.my_method() is
    # run from the global namespace, execution enters a temporary function
    # namespace that is totally separate from the MyClass namespace.

print('Module scope can access ' + MyClass.class_string +
      ' as an attribute of MyClass')
print('Module scope can access {} as an attribute of MyClass'.format(
      MyClass.my_method))
MyClass.my_method()

try:
    print(class_string)
except Exception as e:
    print(e)

# This is useful, intuitive encapsulation! Classes are namespaces, hence are
# containers for variables/objects, which can be dot-accessed as attributes of
# the class.

print('')

# Just to go through one more example, consider this function:
def f():
    print(MyClass.class_string)
f()
# What happened here is that when the f() was called, execution entered a
# temporary namespace for f. It then encountered MyClass and looked in the
# current temporary namespace for the 'MyClass' variable. It was not found, so
# python looked in the parent (global) namespace. It found 'MyClass' there. Then
# since we asked for the 'class_string' attribute of 'MyClass', python looked
# for the 'class_string' variable within the 'MyClass' namespace it had found.

Module scope can access class_string as an attribute of MyClass
Module scope can access <function MyClass.my_method at 0x10518e170> as an attribute of MyClass
MyClass can access class_string as an attribute of MyClass
name 'class_string' is not defined

class_string


# Instances

In [14]:
# Instantiating a class by calling it creates a new variable, an 'instance' of
# the class. The instance is it's own namespace. However, it is not a
# free-floating namespace but is instead a child of the class's namespace, which
# means that if the interpreter cannot find a variable in the instance's
# namespace, it is then refered to the class namespace to look for it there.

# This is not the same as a nested namespace. A nested namespace is a namespace
# within a namespace, like we get with nested modules (e.g. np.linalg.norm).
# We cannot access instance variables through the class namespace like
# MyClass.instance.instance_variable. Instead, think of the instance's child
# namespace as disjoint from the class namespace but with a reference to tell
# the interpreter where to look when it can't find a variable.

%reset -f

class MyClass():
    class_string = 'class_string'
    class_list = ['class_string']
    
    # Note: You can put anything in a class. Most common are functions (which
    # are then called class methods), but it's totally acceptable to put other
    # types of variables in here (strings, arrays, dicts, etc.). Though it's
    # generally frowned upon to put classes in a class, for readability reasons.

# Under the hood, calling the class runs the constructor method __init__(), but
# we aren't doing anything fancy in our class so we don't even bother writing
# __init__(). By the way, all classes secretly inherit from an 'object' type
# that has it's own __init__() constructor that is run when we don't write our
# own __init__(), but that's getting ahead of ourselves!
instance = MyClass()

print('instance can access ' + instance.class_string +
      ' in the class namespace')

# Being a namespace, instance can also have its own variables
instance.instance_string = 'instance_string'
print('instance can access ' + instance.instance_string)

# Variables in instance's namespace are not accessible from the parent (class)
# namespace or grandparent (global) namespace.
try:
    print(MyClass.instance_string)
except Exception as e:
    print(e)

# Danger: Since an instance namespace refers to the parent namespace when a
# variable isn't found, we can unwittingly corrupt a mutable variable in the
# class namespace even if it looks like we're only touching the instance.
other_instance = MyClass()
print(other_instance.class_list)  # Uncorrupted
instance.class_list.pop()
instance.class_list.append('corruption')
print(other_instance.class_list)  # Corrupted

# General opinion: Class attributes are great. In fact, oftentimes I think
# people don't use them enough. However, for safety always make sure they're an
# immutable type so they can't be secretly corrupted by instances.

instance can access class_string in the class namespace
instance can access instance_string
type object 'MyClass' has no attribute 'instance_string'
['class_string']
['corruption']


In [185]:
# Something special happens when you call a method in an instance's namespace:
# The instance itself is passed in as the first argument of that method. For
# example, python interprets this:
#     instance.some_method(*args, **kwargs)
#   as this:
#     instance.some_method(instance, *args, **kwargs)
# This behavior happens even if the method is actually found in the class's
# namespace. This is purely syntactic sugar --- you can avoid it by doing
# Class.some_method(instance, *args, **kwargs). But the syntactic sugar is
# useful because it means you don't have to remember the class name every time
# you want to run a class method on an instance.

%reset -f

class MyClass():
    
    def method_without_arg():
        print('method without argument')
    
    def method_with_arg(arg):
        # Note: There is a convention of calling the instance arg 'self'
        print('method with argument')
    
    # @staticmethod is a decorator to tell python not to do this special
    # behavior of feeding the instance as the first argument.
    @staticmethod
    def static_method_without_arg():
        print('static method without argument')

instance = MyClass()
        
# Calling method_without_arg from the class's namespace is fine
MyClass.method_without_arg()

# Calling method_with_arg from the class's namespace needs an argument
try:
    MyClass.method_with_arg()
except Exception as e:
    print(e)

# Calling method_with_arg from the instance namespace is fine
instance.method_with_arg()

# Calling method_without_arg from the instance namespace gets too many arguments
try:
    instance.method_without_arg()
except Exception as e:
    print(e)

# Calling static_method_without_arg from the instance namespace is fine
instance.static_method_without_arg()

# Calling static_method_without_arg from the class namespace is also fine
MyClass.static_method_without_arg()

# Rule of thumb: If your method does not depend on the instance argument
# ('self') and you want to call it from instances, use the @staticmethod
# decorator! No need to have extra arguments if you don't need them.

method without argument
method_with_arg() missing 1 required positional argument: 'arg'
method with argument
method_without_arg() takes 0 positional arguments but 1 was given
static method without argument
static method without argument


In [186]:
################################################################################
# ASIDE: DECORATORS
################################################################################

# The term 'decorator' refers to a function that takes in one function and
# returns another. For example, f() here is a decorator:
# 
#     def f(g):
# 
#          def _new_g(*args, **kwargs):
#              print(args)
#              print(kwargs)
#              return g(*args, **kwargs)
# 
#          return def _new_g
#
# You can think of a decorator as wrapping up g to make a small modification to
# it, which in this example just prints the args and kwargs of g before running
# g.
# If you want to apply this decorator to a method in a class every time that
# method is called, python has the '@' syntactic sugar, i.e.
#
# class MyClass():
# 
#    @f
#    g(*args, **kwargs):
#        # do whatever g is suppsed to do
#        pass
# 
# This will cause MyClass.g(*args, **kargs) to behave like
# f(MyClass.g)(*args, **kwargs)
#
# Personally, I don't often write my own decorators. In fact, I typically try to
# avoid them for readability reasons --- if I can let the method do whatever it
# needs to do without a decorator, that is usually more straightforward for the
# reader to understand.
# 
# However, in cases when you want to override default Python functionality,
# python has some built-in decorators, like @staticmethod. Other decorators I
# use often are @property to make a class method behave like an attribute, and
# @abc.abstractmethod / @abc.abstractproperty when writing abstract base
# classes.

################################################################################

In [40]:
# The great thing about instances creating child namescopes is we can create
# lots of instance-specific variables in our class methods. The first
# opportunity to do this is in the constructor __init__(), which is called upon
# instance creation.

%reset -f

class MyClass():
    
    def __init__(self, *args, **kwargs):
        # Recall, there's nothing special about using the argument name 'self'
        # here --- it's just convention and the code will work if you use a
        # different name.
        self._args = args
        self._kwargs = kwargs
    
    def print_args(self):
        print(self._args)
    
    def print_kwargs(self):
        print(self._kwargs)

# Remember, calling a class creates a new instance namespace and calls the
# __init__() method, feeding in that new namespace as the first argument
instance_0 = MyClass(1, 2, 3, a=4, b=5)
instance_0.print_args()
instance_0.print_kwargs()

# We can add new attributes to it
instance_0.c = 6
print(instance_0.c)

# We can create lots of instances, and their variables will all be neatly
# encapsulated in their respective namespaces
instance_1 = MyClass('h', 'i', x=True, y=(1, 2, 3))
instance_1.print_args()
instance_1.print_kwargs()

(1, 2, 3)
{'a': 4, 'b': 5}
6
('h', 'i')
{'x': True, 'y': (1, 2, 3)}


# Inheritance

We talked about these things:
* Classes are namespaces
* Instances are namespaces
* Instance namespaces are children of class namespaces, i.e. they refer to the
  class namespace if the interpreter can't find a variable in them.

These suggest that if we want one class to have access to methods from another
class, we somehow need to make that class's namespace be a child of the other's
namespace. Then instances of the child class will be grandchild namespaces of
the parent class. That is what inheritance does.

In [188]:
# Parent/child classes, no instances here yet.

%reset -f

class Parent():
    def parent_method():
        print('parent method')
        
class Child(Parent):
    def child_method():
        print('child method')
        
# Now Child has access to Parent's methods:
Child.parent_method()

# But Parent does not have access to Child's methods:
try:
    Parent.child_method()
except Exception as e:
    print(e)

parent method
type object 'Parent' has no attribute 'child_method'


In [189]:
# Consequently, child class instances likewise inherit methods from the parent
# class (the parent class namespace is the grandparent of the instance
# namespace).

%reset -f

class Parent():
    parent_string = 'parent_string'
    
    def parent_instance_method(self):
        print('parent instance method')
        
class Child(Parent): pass
        
child_instance = Child()
print(child_instance.parent_string)
child_instance.parent_instance_method()

# Note: Child classes can implement variables that have the same name as
# variables in their parents. If this is the case, the interpreter never gets
# referred to the parent when looking these up, so that variable has been
# effectively overridden in the child class.

class UnrulyChild(Parent):
    def parent_instance_method(self):
        print('I have overridden my parent')
        
unruly_child_instance = UnrulyChild()
unruly_child_instance.parent_instance_method()

parent_string
parent instance method
I have overridden my parent


In [109]:
# Multiple Inheritance: Python does allow multiple inheritance. In this case,
# python searches parent namespaces left-to-right when looking up a variable.
# This is called Method Resolution Order (MRO).

%reset -f

class Parent0():
    parent_string = 'parent string 0'
    
class Parent1():
    parent_string = 'parent string 1'
        
class Child(Parent0, Parent1): pass
        
print(Child.parent_string)

# Rule of thumb. Multiple inheritance is a bad idea. It requires cognitive
# overhead for the programmer & reader to know what variables are in the
# parents to figure out if there are duplicates, and can give rise to gnarly
# bugs. Things get even more messy when grandparents get involved ...

parent_string_0


In [191]:
# Multiple Inheritance with Grandparents: A complete mess.

%reset -f

class GrandParent():
    grandparent_string = 'grandparent string'

class Parent0(GrandParent): pass

# Let's try making Parent1 inherit from GrandParent too, so we have a
# diamond-shaped inheritance structure. In this case, the method resolution
# order is Child -> Parent0 -> Parent1 -> GrandParent, as we can see:

class Parent1(GrandParent):
    grandparent_string = 'overridden grandparent string'
        
class Child(Parent0, Parent1): pass
        
print(Child.grandparent_string)  # prints 'overridden grandparent string'

# From this it might seem like python is doing a breadth-first search. However,
# that is actually not quite the case:

# Let's try making Parent1 not inherit from GrandParent:

class Parent1():
    grandparent_string = 'overridden grandparent string'
        
class Child(Parent0, Parent1): pass
        
print(Child.grandparent_string)  # prints 'grandparent string'

# Now the method resolution order is
#     Child -> Parent0 -> GrandParent -> Parent 1.
# This is strange --- while the MRO looked like breadth-first search when we
# had a diamonnd in the structure, removing one edge of that diamond makes it
# now look like depth-first search.

# You can imagine how this could cause really devious bugs in practice and be
# really hard to keep track of as a programmer/reader.

# To make matters worse, this is just one of a large number of counterintuitive
# behaviors in python's MRO algorithm (things get even crazier when you add
# deeper/wider inheritance structures). Moreover, python's MRO algorithm changed
# between python2 and python3, and a lot of programmers (myself included) got
# confused by that, in part because we didn't fully understand the MRO algorithm
# in python2 to being with!

# Bottom line: Avoid multiple inheritance whever possible. If you absolutely
# must use multiple inheritance, be sure to check carefully for variables with
# the name in the parent lineages and document the inheritance structure well.

# Most python programmers flatly refuse to use multiple inheritance unless one 
# of the parents is a "Mixin" --- a parent class that has no __init__() function
# and is very light-weight.

overridden grandparent string
grandparent string


# super(): Invoking a parent namespace.

Sometimes you may want to get a variable in a parent namespace that has the same
name as a variable in the child namespace. In this case you can't just get the
variable as usual because the interpreter will pick up the one in the child
namespace. super() allows you to explicitly tell the interpreter to jump to the
parent namespace and begin its search there.

In [42]:
# Here is basic super() usage:

%reset -f

class Parent():
    class_string = 'parent'
    
class Child(Parent):
    class_string = 'child'
    
    def print_string(self):
        # Within an instance method, super() returns the parent namespace
        print(super().class_string)

child_instance = Child().print_string()  # prints 'parent'

# Note that we don't need super() for this --- we could invoke Parent by name,
# like this:

class Child(Parent):
    class_string = 'child'
    
    def print_string(self):
        print(Parent.class_string)

Child().print_string()  # prints 'parent'

# However, using super() is slightly preferable because it makes the code more
# robust. For example, if you decide to change the name of Parent, with super()
# you don't have to go through the entire Child implmementation changing all the
# times Parent was invoked by name.

parent
parent


In [134]:
# super() is only needed when you have variables that share the same name in the
# child and parent classes (otherwise you can use normal inheritance without
# super()). This is often used when you want the child class method to implement
# a wrapper of the parent class method. One very common occurance is in
# __init__(), for example:

%reset -f

class Parent():
    def __init__(self, **parent_kwargs):
        # Does a bunch of useful stuff that you want children to do too
        pass
    
    
class Child(Parent):
    def __init__(self, child_arg, **parent_kwargs):
        # Do the useful stuff in Parent's constructore
        super().__init__(**parent_kwargs)
        
        # Do a little extra for the child
        self._child_arg = child_arg

# As you might imagine, calling super() when there is multiple inheritance gets
# into python's method resolution order, which is a mess and should be avoided.

In [193]:
# super() can optionally take two arguments:
#     super(class, instance)
# where 'class' is the class who's parent we want to invoke and 'instance' is an
# instance of that class.

# One situation when when you might want to use these arguments is using super()
# outside of a class method, e.g. later on in the code, like this:

%reset -f

class Parent():
    class_string = 'parent'
    
class Child(Parent):
    class_string = 'child'

child_instance = Child()
print(super(Child, child_instance).class_string)

# Another is if you want to invoke a grandparent class, like this:

class GrandParent():
    class_string = 'grandparent'

class Parent(GrandParent):
    class_string = 'parent'

class Child(Parent):
    class_string = 'child'
    
    def print_string(self):
        print(super(Parent, self).class_string)

Child().print_string()  # prints 'grandparent'

parent
grandparent


# Dunder Methods

In [152]:
# Double underscore (or "dunder") methods are private class methods that python
# calls according to particular rules.
# For example, __init__() is one, which is called when an instance is created.
# Another example is __str__, which is called by the 'str()' python function.

# There are actually a lot of dunder methods, even for an object as simple as
# the integer 4:
print([x for x in dir(4) if '__' in x])
# Most of these you rarely if ever need to worry about, but they do a lot of
# work behind the scenes. For example, when you type 3 < 4, python actually
# calls the __lt__() dunder, i.e. 3.__lt__(4).

# Always take care when implementing or overriding dunder methods --- they're
# double-underscore protected for a reason. __init__() is by far the most common
# to override, but sometimes you may want to override others. In the next few
# cells are the ones I implement/override most often:

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__']


In [153]:
# __call__(): This method is called whenever you call something. In fact, any
# time you create a function (e.g. def f(): ...), that function is actually an
# object whose __call__() method is what you wrote! Unlike functions, classes
# are not by default callable, so I sometimes make a class callable by
# implementing __call__(). For example:

%reset -f

class CallableClass():
    
    def __init__(self, *args):
        self._args = args

    def __call__(self):
        print(self._args)
        
CallableClass(1, 2, 3)()

# Making callable classes can actually be quite useful, because a callable class
# can behave like a function, hence can fed into calling code that expects a
# function.

(1, 2, 3)


In [195]:
# __str__(): This method is called whenever you call str() on something. This
# one is particularly useful when you want to make logging and/or error strings
# easier to read or more informative.

%reset -f

class CallableClass():
    
    def __init__(self, *args):
        self._args = args
        

class PrettyCallableClass():
    
    def __init__(self, *args):
        self._args = args
        
    def __str__(self):
        return '<Callable with args {}>'.format(self._args)


print(CallableClass(1, 2, 3))
print(PrettyCallableClass(1, 2, 3))

<Callable with args (1, 2, 3)>


In [197]:
# Okay I'll stop writing examples now but will just list a few others:

# __len__(): This method is called whenever you call len() on something.

# __hash__(): This is called whevener you call hash() on something.

# __setattr__(): This sets an attribute. For example,
#     A.name = value   does   A.__setattr__(name, value)

# __setitem__(): This sets an item. For example,
#     A[name] = value   does   A.__setitem__(name, value)

# __bool__(): Called whenever you call bool() on something.

# __enter__() and __exit__(): These are the dunders you have to implement for
# context managers.

# Private class attributes

Python doesn't have truly private and public class attributes --- as long as you
have an attribute name, you can look it up in the class namescope.

However, there is a convention of using a single leading underscore in a class
attribute to indicate the attribute should not be used publicly. This is not
enforced --- it is purely convention. By the way, this applies to functions as
well --- if you want a function to be private to a module, use a leading
underscore in its name.

There is also name mangling, which goes a step further: If a class attribute has
two leading underscores in its name, python will alias it by a mangled version
of its name (specifically, _ClassName__attribute_name). It still isn't truly
private (you can still access it publicly via its mangled name), but the name
mangling makes it very apparent that it is supposed to be private.

By the way, if an attribute has both leading and trailing double underscores, it
it exempt from name mangling (e.g. magid dunder methods like __init__ and
__call__).

Typically, I use single underscore attributes all the time but rarely use double
underscore attributes. As long as I abide by the convention of not accessing
single underscore attributes publicly, there isn't really a need to use name
mangling. I'll only use double underscores if I'm going to open-source the code
and want to really force privacy for robustness.

In [199]:
# Single underscore private method convention

%reset -f

class MyClass():
    _private_string = 'private string'
    
    def _private_method():
        print('private method')
        
# Single underscore is just a convention --- it can still be accessed:
print(MyClass._private_string)
MyClass._private_method()

# However, most linters will give you a warning if you try to access a single
# underscore attribute, and you should try to avoid doing so.

private string
private method


In [204]:
# Double underscore name mangling

%reset -f

class MyClass():
    __private_string = 'private string'
    
    def __private_method():
        print('private method')
        
    def print_private_string():
        # No name mangling when accessed from within the class
        print(MyClass.__private_string)
        
    def print_private_string_from_instance(self):
        # Whether or not instances are involved:
        print(self.__private_string)


try:
    print(MyClass.__private_string)
except Exception as e:
    print(e)

try:
    MyClass.__private_method()
except Exception as e:
    print(e)

# Here's how the name mangling works:
print(MyClass._MyClass__private_string)
MyClass._MyClass__private_method()

# Accessing from within the class has no name mangling
MyClass.print_private_string()
MyClass().print_private_string_from_instance()

type object 'MyClass' has no attribute '__private_string'
type object 'MyClass' has no attribute '__private_method'
private string
private method
private string
private string


# @property Decorator

Oftentime you may want an attribute to be half private in the sense that you
want it to be accessible publicly but you don't want it to be able to be
overwritten publicly.

One way to handle this is to have a private attribute but implement a public
getter method that returns the private attribute. This is fine, but sometimes
the calling code would be more intuitive if it didn't have to call a getter
method.

This is where the @property decorator comes into play. The @property decorator
makes a method behave like an attribute, i.e. it allows the caller to call the
method without actually calling it.

In [206]:
# Making an attribute publicly accessible by not publicly writable.

%reset -f

# Method 1: Make a getter method for a private attribute

class MyClass():
    _private_string = 'private string'
    
    def private_string_getter(self):
        return self._private_string

print(MyClass().private_string_getter())

# Method 2: Use @property decorate to make getter behave like an attribute

class MyClass():
    _private_string = 'private string'
    
    @property
    def private_string(self):
        return self._private_string
    
    @property
    def random_property(self):
        return 42
    
print(MyClass().private_string)
print(MyClass().random_property)

# By the way, this would work similarly if you were using double underscore
# attributes.

private string
private string
42


In [207]:
# By the way, it is a very common to use single underscore private attributes
# and make them publicly accessible (but not publicly writable by convention)
# with @property methods. Those @property methods are customarily put at the
# bottom of the class implementation, like this:

%reset -f

class MyClass():
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
    
    def my_method(self):
        return self._a + self._b
    
    @property
    def a(self):
        return self._a
    
    @property
    def b(self):
        return self._b
    
    @property
    def c(self):
        return self._c

# Setters

The python @x.setter decorator allows you to write some functionality to be done
whenever a variable x is set.

This can be useful if you have a private attribute that you later decide should
be publicly settable (but don't want to or can't change the private attribute's
name everywhere).

This is also just a generally useful tool --- I regularly find myself wanting to
have a method that sets an attribute and does other things simultaneously, and
the @x.setter syntactical sugar often makes this more intuitive for the calling
code than making some method with a new name to do that.

In [221]:
# Setters

%reset -f

class MyClass():
    def __init__(self):
        self._private_string = 'private string'

    @property
    def private_string(self):
        print('getting private_string')
        return self._private_string
    
    @private_string.setter
    def private_string(self, new_value):
        print('setting private_string')
        self._private_string = new_value
        

instance = MyClass()
instance.private_string = 'new'
print(instance.private_string)

setting private_string
getting private_string
new
